In [41]:
import spacy

nlp = spacy.load("nl_core_news_lg")

In [42]:
from tqdm import tqdm

import gzip
import os

import random

import pandas

corpus = "./Corpus/"
anns = "./Annotations/"

def load(file, corpus, lines=False):
    with open(os.path.join(corpus, file), "r") as handle:
        if lines:
            return list(handle) # handle.readlines()
        else:
            return handle.read()

In [46]:
def read_csv(file, corpus=anns):
    file = load(file, corpus, lines=True)
    for line in file:
        tab_cols = line.split("\t")
        
        if tab_cols[0].startswith("T"):
            last = tab_cols[-1].strip()
            first = tab_cols[0]
            
            middle = tab_cols[1].split()
#             print(middle, middle[0], middle[-1])
            middle = [middle[0], middle[1], middle[-1]]
            
            yield (first, *middle, last)
            
def filter_rows(rows):
    for r in rows:
        if r[1].upper() in ["PERSON", "ORG", "GPE"]:
            yield r
            
def change_rows(rows):
    for r in rows:
        entity_nr, entity_type, start, end, label = r
        yield int(start), int(end), entity_type.upper()

In [47]:
data = {}
for ann_f in os.listdir(anns):
    if len(load(ann_f, anns)) > 0:
        cur_name = ann_f.strip(".ann")
    
        txt_f = cur_name + ".txt"
        if os.path.isfile(os.path.join(corpus, txt_f)):
        
            raw_text = load(txt_f, corpus)
            brat_entity_list = list(change_rows(filter_rows(read_csv(ann_f, anns))))
            spacy_entity_list = [(e.start_char, e.end_char, e.label_) for e in nlp(raw_text).ents]

            data[cur_name] = (raw_text.replace("\n", " "), brat_entity_list, spacy_entity_list)
        else:
            print("no file", txt_f)
    else:
        print("annotation file empty")

no file gitkeep.txt


In [25]:
import random

def range_in(ent, ent_ls):
    for ent2 in ent_ls:
        x = ent[:-1]
        y = ent2[:-1]
        if len(range(max(x[0], y[0]), min(x[-1], y[-1])+1)) > 0:
            return True
    return False
        
def merge(b_ls, s_ls):
    l = len(b_ls)
    
    l = len(s_ls)//1
    k = 0
    for s_ent in random.sample(s_ls, len(s_ls)):
        if k == l:
            break
        if not range_in(s_ent, b_ls):
            yield s_ent
            k += 1
            
    yield from b_ls
            
    
    
    
    
data_merged = {}

for f, (txt, b_ls, s_ls) in data.items():    
    ents = sorted(merge(b_ls, s_ls), key=lambda tup: tup[0])
    
    data_merged[f] = (txt, ents)

In [16]:
data_merged


{'NL-HaNA_1.04.02_6847_0016': ('Zij dat dit instrument na een hunner verder overlijden van volk„ „men krogt en Effect wesen zal ¶ zij als testament Codicell of zodanige t zelve best na regten zal konnen bestaan schoon Eenige Solemnityten ten desen gerequireerd werden alle met na behooren mogten wesen g’obserteerd, Aldus gedaan ende gepasseerd ten huijze van de Testateuren van staande of de tijgersgragt binnen dese stad ter presentie Clercq: Abraham Salomons en Hendrik Abrahamsz als getriggen De minute deses is behoorlijk getekend en gesz: op een Zegul van Een Ed: /onderstond/ gnodaterston was getek:d Eggbert Blomhert /mots: naar gedaane Collatie is dese met dies origineel bevonden te accordeeren by my bovengenoemde nots alhier resideerende Natavo den 13: 8bert 1768: Marten Albregt Heynagtez N„o 3 op Heeden den 15: November 1768, Compareerde voor mij m:r Egbert Blomhert nots: pnbl: by de Edele hooge regeringe van nederlands Jordia G’admitteerd binnen de stad batera resideerende inde voo